In [1]:
import AVFoundation
import CoreAudio
import CoreMediaIO
import struct
import json


vopa = CoreMediaIO.CMIOObjectPropertyAddress( 
    CoreMediaIO.kCMIODevicePropertyDeviceIsRunningSomewhere 
)

aopa = CoreAudio.AudioObjectPropertyAddress(
    CoreAudio.kAudioDevicePropertyDeviceIsRunningSomewhere,
    CoreAudio.kAudioObjectPropertyScopeGlobal,
    CoreAudio.kAudioObjectPropertyElementMaster
)

In [2]:
device_types = [
    AVFoundation.AVCaptureDeviceTypeBuiltInWideAngleCamera,
    AVFoundation.AVCaptureDeviceTypeContinuityCamera,
    AVFoundation.AVCaptureDeviceTypeExternal,  # Covers external cameras
]

discovery_session = (
    AVFoundation.AVCaptureDeviceDiscoverySession.discoverySessionWithDeviceTypes_mediaType_position_(
        device_types,
        AVFoundation.AVMediaTypeVideo,
        AVFoundation.AVCaptureDevicePositionUnspecified,
        )
    )

devices = discovery_session.devices()

for device in devices :
    print( device.connectionID(), device.localizedName() )



2024-08-23 23:16:54.170 Python[49333:83203881] WARNING: Add NSCameraUseContinuityCameraDeviceType to your Info.plist to use AVCaptureDeviceTypeContinuityCamera.


39 FaceTime HD Camera
35 Fataffe Camera


2024-08-23 23:16:54.497 Python[49333:83203881] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


In [5]:

mic_ids = {
    mic.connectionID(): mic
    for mic in AVFoundation.AVCaptureDevice.devicesWithMediaType_(
        AVFoundation.AVMediaTypeAudio
    )
}

aresponse = []
for mic_id in mic_ids:
    response = CoreAudio.AudioObjectGetPropertyData(mic_id, aopa, 0, [], 4, None)
    try :
        aresponse.append(
            {
                "Mic" : str(mic_ids[mic_id].localizedName()),
                "Active" : struct.unpack('I', response[2])[0]
            }
        )
    except Exception as Err :
        aresponse.append(
            {
                "Mic" : str(mic_ids[mic_id].localizedName()),
                "Active" : None
            }
        )


In [6]:
vid_ids = {
    vid.connectionID(): vid
    for vid in AVFoundation.AVCaptureDevice.devicesWithMediaType_(
        AVFoundation.AVMediaTypeVideo
    )
}

vresponse = []
for (did, cam) in vid_ids.items() :
    response = CoreMediaIO.CMIOObjectGetPropertyData( did, vopa, 0, None, 4, None, None )
    try :
        vresponse.append(
            {
                "Cam" : str(cam.localizedName()),
                "Active" : struct.unpack('I', response[3])[0]
            }
        ) 
    except Exception as Err :
        vresponse.append(
            {
                "Cam" : str(cam.localizedName()),
                "Active" : None
            }
        ) 


In [7]:
print( json.dumps({ "Audio" : aresponse, "Video" : vresponse}, indent=4))

{
    "Audio": [
        {
            "Mic": "Fataffe Microphone",
            "Active": null
        },
        {
            "Mic": "MacBook Pro Microphone",
            "Active": 0
        },
        {
            "Mic": "External Microphone",
            "Active": 1
        },
        {
            "Mic": "Microsoft Teams Audio",
            "Active": null
        }
    ],
    "Video": [
        {
            "Cam": "FaceTime HD Camera",
            "Active": 1
        },
        {
            "Cam": "Fataffe Camera",
            "Active": 0
        }
    ]
}
